# Writing Data

The same bracket syntax used for reading also supports writes.
Values are sent to PowerWorld immediately.

```python
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load

wb = GridWorkBench("path/to/case.pwb")
```

In [7]:
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch
import numpy as np
import pandas as pd
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 8.8870 sec


The indexable interface supports four write patterns:

| Syntax | Behavior |
|---|---|
| `wb[Gen, "GenMW"] = 100.0` | Broadcast a **scalar** to every object |
| `wb[Gen, "GenMW"] = [100, 150, ...]` | Set **per-element** values (length must match) |
| `wb[Gen, ["GenMW", "GenStatus"]] = [100, "Closed"]` | Broadcast to **multiple fields** at once |
| `wb[Bus] = df` | **Bulk update** from a DataFrame (must include key columns) |

## Scalar Broadcast

A single value is applied to every object of that type.

In [8]:
wb[Gen, "GenMW"] = 100.0

## Per-Element Values

A list or array whose length matches the number of objects sets
each one individually.

In [9]:
wb[Gen, "GenMW"] = np.linspace(50, 200, len(wb[Gen]))

## Multiple Fields

Set several fields at once by passing a list of field names and a
matching list of values.

In [10]:
wb[Gen, ["GenMW", "GenStatus"]] = [100.0, "Closed"]

## DataFrame Update

For targeted updates to specific objects, build a DataFrame that
includes the primary-key columns and the fields to change.

In [11]:
updates = pd.DataFrame({
    "BusNum": wb[Bus]["BusNum"].head(3),
    "BusPUVolt": [1.02, 1.01, 0.99]
})
wb[Bus] = updates

## Read-Modify-Write

A common workflow: read existing values, transform them in pandas,
and write back. Here we scale all loads by 10%.

In [12]:
loads = wb[Load, ["LoadMW", "LoadMVR"]]
loads["LoadMW"] *= 1.10
loads["LoadMVR"] *= 1.10
wb[Load] = loads